In [ ]:
# packages

import numpy as np
import pandas as pd
import datetime
#from datetime import datetime
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

In [ ]:
# data
DATA_DIR = '/media/bigdata/10. Stages/3. Afgerond/2020-08 Jesse Kuiper/'

In [ ]:
# load opnamens
opnamen = pd.read_csv(DATA_DIR + "werkbestanden-opnames/latest/werkbestand_opnames.csv", sep=';')

In [ ]:
# load administered
Toedienen = pd.read_csv(DATA_DIR + "werkbestanden-medicatie/latest/werkbestand_medicatie_toediening.csv", sep=';',decimal=',' )

In [ ]:
# load dbc
dbcgeheel = pd.read_csv(DATA_DIR + "werkbestanden-dbc/latest/werkbestand_dbc.csv", sep=';')

In [ ]:
# load map
boos = pd.read_csv(DATA_DIR + "werkbestanden-map/latest/werkbestand_map.csv", sep=';')

In [ ]:
#load patient or patient uniek
pat = pd.read_csv(DATA_DIR + "werkbestanden-patient/latest/werkbestand_patient_uniek.csv", sep=';')

In [ ]:
# data selection

In [ ]:
# selecting usefull var 
# what to do with "AfdelingOmschrijving" - just the most occuring ones
admission = opnamen[["OpnameID", "PseudoID","AfdelingOmschrijving", "Opnamedatum",  "Ontslagdatum",  "Opnametijd", 
"Ontslagtijd",    "Spoed",  "EersteOpname",  "Geslacht",
"Leeftijd_opname", "OpnamestatusOmschrijving", "Duur",]]

administering = Toedienen[["PseudoID", "VoorschriftID", "ATC_code_omschr", "Medicijnnaam_ingevuld","Dosis", "Eenheid", "ToedienDatum", "ToedienTijd", "Toegediend", "DosisVerbruikt", "DosisOrigineel", "ToedieningIsOpgeschort", "NietToegediend"  ]]

dbc = dbcgeheel[[ "PseudoID", "dbcnummer","Startdatum", "Einddatum" ,"hoofddiagnose_groep", "zorgvraagzwaarte", "MeervoudigeProblematiekInd",
                            "persoonlijkheidsstoornis", "Opname", "DiagnoseDatum" ]]

violent = boos[["PseudoID", "hantering_datum", "begin_incident",]]


patient = pat[["PseudoID", "Leeftijd_startdatum_dossier" ]]

In [ ]:
##############################################################################################################

In [ ]:
# check for na values
admission.isnull().sum()

In [ ]:
admission['AfdelingOmschrijving'].value_counts()

In [ ]:
# drop na values in admission
# remove incomplete admissions 
admission = admission[admission.OpnamestatusOmschrijving == "Ontslagen"]

# change Opnamedatum Ontslagdatum to date times        
admission["OpnamedatumTijd"] = pd.to_datetime(admission["Opnamedatum"] + ' ' + admission["Opnametijd"])
admission["OntslagdatumTijd"] = pd.to_datetime(admission["Ontslagdatum"] + ' ' + admission["Ontslagtijd"])

# DateTime checks for the agression and the dbc
# these datetimes make sure it only covers the 
# If the duration of admission is less than the time check, it will take the whole admission
admission["DaysF"] = np.where(admission["Duur"]>= 14, 14, admission["Duur"])
admission["DaysP"] = np.where(admission["Duur"]>= 3, 3, admission["Duur"])

# create date time checks #these should have a max value
admission["DateTimeCheckF"] = admission["OpnamedatumTijd"] + pd.to_timedelta(admission["DaysF"], unit='d')
admission["DateTimeCheckP"] = admission["OpnamedatumTijd"] + pd.to_timedelta(admission["DaysP"], unit='d')

# create date time checks + 14 days
#admission["DateTimeCheckFTwoWeeks"] = admission["DateTimeCheckF"] + 



afd = ["Klin. Affectieve & Psychotische stoorn.","Klinische Acuut & Intensieve Zorg","Klin.Acuut & Intensieve. Zorg Jeugd", "Klin Diagn & Vroege Psychose"]

# AfdelingOmschrijving
adm_afd = pd.get_dummies(admission["AfdelingOmschrijving"])
adm_afd_sel = pd.concat([admission, adm_afd[afd]], axis=1)

del adm_afd_sel["AfdelingOmschrijving"]

adm_afd_sel

In [ ]:
admission.dtypes

In [ ]:
admission.isnull().sum()

In [ ]:
# create admission1 - 8663, admission2 - 3192 and admission3 - 4685
admission1 = adm_afd_sel.copy()
admission2 = adm_afd_sel[admission["Duur"]> 14].reset_index().copy()
admission3 = adm_afd_sel[admission["Duur"]> 3].reset_index().copy()

In [ ]:
# check na values in dbc
dbc.isnull().sum()

In [ ]:
# drop na values in the dbc
# select only dbc from opnamens
# dbc = dbc[dbc.Opname == "Ja"]
# PABLO: I wonder: is this necessary? If you don't do it, and you merge on PseudoID and date, what happens?

# change NaN in hoofddiagnose_groep to "Lege hoofddiagnose" as this is already a variable in the table with the same meaning
dbc["hoofddiagnose_groep"] = dbc["hoofddiagnose_groep"].replace(np.nan, "Lege hoofddiagnose", regex=True)
dbc["hoofddiagnose_groep"] = dbc["hoofddiagnose_groep"].str.replace("Bijkomende codes/geen diagnose","Lege hoofddiagnose")

# create a diagnose date
# this is a limitation to be mentioned in the paper
def get_diagnosis_date(row):
    if type(row.DiagnoseDatum) == str:
        return row.DiagnoseDatum
    elif type(row.Einddatum) == str:
        return row.Einddatum
    else:
        return row.Startdatum

dbc["diagnosis_date"] = dbc.apply(lambda row: get_diagnosis_date(row), 1)

dbc.isnull().sum()

In [ ]:
violent.isnull().sum()

In [ ]:
# drop na values violent
violent = violent.dropna(subset=["PseudoID"])

# change hantering_datum to date time with begin_incident
violent["hantering_datumTijd"] = pd.to_datetime(violent["hantering_datum"] + ' ' + violent["begin_incident"])

violent.isnull().sum()

In [ ]:
# check for NaN in patient
patient.isnull().sum()

In [ ]:
administering.isnull().sum()

In [ ]:
# we are only interested in administered medicine
administering = administering[administering["Toegediend"]==1]

# list of agreed upon tranquilizers
AllTranq = ["lorazepam", "oxazepam","diazepam", "zopiclone", "temazepam" , "midazolam" , "zolpidem" , "alprazolam",
       "bromazepam", "lormetazepam", "clorazepate potassium", "nitrazepam", "flurazepam", "clobazam", "chlordiazepoxide", "brotizolam"]

administering = administering[administering["ATC_code_omschr"].isin(AllTranq)]

administering.isnull().sum()

In [ ]:
# 1 administering does not contain a toediendatum and toedientijd (corrupted data)
administering = administering.dropna(subset=["ToedienDatum"])
administering.isnull().sum()

In [ ]:
# create datetime
administering["ToedienDatumTijd"] = pd.to_datetime(administering["ToedienDatum"] + ' ' + administering["ToedienTijd"])

administering.isnull().sum()

In [ ]:
# merge administering
# based on the dosis
TranqDict = {
    "lorazepam": 5,
    "oxazepam": 0.33,
    "diazepam": 1,
    "zopiclone": 1.33,
    "temazepam": 1,
    "midazolam": 10,
    "zolpidem": 1,
    "alprazolam": 10,
    "bromazepam": 1,
    "lormetazepam": 10,
    "clorazepate potassium": 0.75,
    "nitrazepam": 1,
    "flurazepam": 0.33,
    "clobazam": 0.5,
    "chlordiazepoxide": 0.5,
    "brotizolam": 40,
}

def InDiazepam(row):
    tranquilizer_multiplier = TranqDict
    omschr = row['ATC_code_omschr']
    dosis = row['Dosis']
    return tranquilizer_multiplier[omschr] * dosis

In [ ]:
administering["DoseDiazepam"] = administering.apply(InDiazepam, axis=1)

administering.isnull().sum()

In [ ]:
###############################################################################################################

In [ ]:
# data set base
# prediction over the course of all the completed cases

In [ ]:
# first add the patient data to the opnamens
def get_adm_pat(frame ):
    adm_adm1 = frame.merge(patient, on="PseudoID", how="left")
    assert np.sum(adm_adm1.isnull().sum().values) == 0
    assert len(adm_adm1) == len(frame)
    return adm_adm1    

In [ ]:
# merge patient data with admission data
adm_pat1 = get_adm_pat(admission1)  

adm_pat2 = get_adm_pat(admission2)

adm_pat3 = get_adm_pat(admission3)

In [ ]:
# merge map data, this is where the datasets start to differ
# give map data an unique identifier
violent["IncidentID"] = np.arange(len(violent))

In [ ]:
# merge dataset with mapdata
map_adm1 = adm_pat1.merge(violent[["PseudoID", "IncidentID", "hantering_datumTijd"]], how="left", on="PseudoID")

In [ ]:
# whole dataset
opname_ids, incidents_during_admission, incidents_before_admission = [], [], []
for opname_id, grp in map_adm1.groupby("OpnameID"):
    # opname_id -> single OpnameID from adm_map table                                                                                                                                                      
    # grp -> a dataframe containing only the rows that have OpnameID == opname_id                                                                                                                          
    opname_ids.append(opname_id)
    if len(grp) == 1 and len(grp[grp["IncidentID"].notnull()]) == 0:
        ''' PABLO: you can remove the condition len(grp) == 1. 
                What you need is groups for which there are no non-null incident IDs'''
        # No incidents                                                                                                                                                                                     
        incidents_during_admission.append(0)
        incidents_before_admission.append(0)
    else:
        # At least one incident                                                                                                                                                                                                                                                                                                                                    
        n_during = len(grp[grp.apply(lambda row: row.OntslagdatumTijd >= row.hantering_datumTijd and row.OpnamedatumTijd <= row.hantering_datumTijd, 1)])
        n_before = len(grp[grp.apply(lambda row: row.hantering_datumTijd <= row.OpnamedatumTijd, 1)])
        incidents_during_admission.append(n_during)
        incidents_before_admission.append(n_before)

# Create a new incidents-counts dataframe
admission_incidents = pd.DataFrame()
admission_incidents["OpnameID"] = opname_ids
admission_incidents["incidents_during_admission"] = incidents_during_admission
admission_incidents["incidents_before_admission"] = incidents_before_admission
assert len(admission_incidents) == len(adm_pat1)

# Merge the incidents-counts dataframe onto opnames
adm_map1 = adm_pat1.merge(admission_incidents, on="OpnameID", how="inner")
assert len(adm_map1) == len(adm_pat1)

In [ ]:
#adm_map1
adm_map1.isnull().sum()

In [ ]:
map_adm2 = adm_pat2.merge(violent[["PseudoID", "IncidentID", "hantering_datumTijd"]], how="left", on="PseudoID")

In [ ]:
#  dataset2 
opname_ids, incidents_during_admission, incidents_before_admission = [], [], []
for opname_id, grp in map_adm2.groupby("OpnameID"):
    # opname_id -> single OpnameID from adm_map table                                                                                                                                                      
    # grp -> a dataframe containing only the rows that have OpnameID == opname_id                                                                                                                          
    opname_ids.append(opname_id)
    if len(grp) == 1 and len(grp[grp["IncidentID"].notnull()]) == 0:
        ''' PABLO: you can remove the condition len(grp) == 1. 
                What you need is groups for which there are no non-null incident IDs'''
        # No incidents                                                                                                                                                                                     
        incidents_during_admission.append(0)
        incidents_before_admission.append(0)
    else:
        # At least one incident                                                                                                                                                                                                                                                                                                                                    
        n_during = len(grp[grp.apply(lambda row: row.DateTimeCheckF >= row.hantering_datumTijd and row.OpnamedatumTijd <= row.hantering_datumTijd, 1)])
        n_before = len(grp[grp.apply(lambda row: row.hantering_datumTijd <= row.OpnamedatumTijd, 1)])
        incidents_during_admission.append(n_during)
        incidents_before_admission.append(n_before)

# Create a new incidents-counts dataframe
admission_incidents = pd.DataFrame()
admission_incidents["OpnameID"] = opname_ids
admission_incidents["incidents_during_admission"] = incidents_during_admission
admission_incidents["incidents_before_admission"] = incidents_before_admission
assert len(admission_incidents) == len(adm_pat2)

# Merge the incidents-counts dataframe onto opnames
adm_map2 = adm_pat2.merge(admission_incidents, on="OpnameID", how="inner")
assert len(adm_map2) == len(adm_pat2)

In [ ]:
#adm_map2
adm_map2.isnull().sum()

In [ ]:
map_adm3 = adm_pat3.merge(violent[["PseudoID", "IncidentID", "hantering_datumTijd"]], how="left", on="PseudoID")

In [ ]:
#  dataset3 
opname_ids, incidents_during_admission, incidents_before_admission = [], [], []
for opname_id, grp in map_adm3.groupby("OpnameID"):
    # opname_id -> single OpnameID from adm_map table                                                                                                                                                      
    # grp -> a dataframe containing only the rows that have OpnameID == opname_id                                                                                                                          
    opname_ids.append(opname_id)
    if len(grp) == 1 and len(grp[grp["IncidentID"].notnull()]) == 0:
        ''' PABLO: you can remove the condition len(grp) == 1. 
                What you need is groups for which there are no non-null incident IDs'''
        # No incidents                                                                                                                                                                                     
        incidents_during_admission.append(0)
        incidents_before_admission.append(0)
    else:
        # At least one incident                                                                                                                                                                                                                                                                                                                                    
        n_during = len(grp[grp.apply(lambda row: row.DateTimeCheckP >= row.hantering_datumTijd and row.OpnamedatumTijd <= row.hantering_datumTijd, 1)])
        n_before = len(grp[grp.apply(lambda row: row.hantering_datumTijd <= row.OpnamedatumTijd, 1)])
        incidents_during_admission.append(n_during)
        incidents_before_admission.append(n_before)

# Create a new incidents-counts dataframe
admission_incidents = pd.DataFrame()
admission_incidents["OpnameID"] = opname_ids
admission_incidents["incidents_during_admission"] = incidents_during_admission
admission_incidents["incidents_before_admission"] = incidents_before_admission
assert len(admission_incidents) == len(adm_pat3)

# Merge the incidents-counts dataframe onto opnames
adm_map3 = adm_pat3.merge(admission_incidents, on="OpnameID", how="inner")
assert len(adm_map3) == len(adm_pat3)

In [ ]:
#adm_map3
adm_map3.isnull().sum()

In [ ]:
# merge dbc

In [ ]:
#function to merge adm_dbc with the right dbc

hoofddiagnoses = [
    "Aandachtsstoornis",
    "Andere problemen die een reden van zorg kunnen zijn",
    "Angststoornissen",
    "Autismespectrumstoornis",
    "Bipolaire stoornissen",
    "Cognitieve stoornissen",
    "Depressieve stoornissen",
    "Dissociatieve stoornissen",
    "Gedragsstoornissen",
    "Middelgerelateerde en verslavingsstoornissen",
    "Obsessieve-compulsieve en verwante stoornissen",
    "Overige psychische stoornissen",
    "Overige stoornissen op zuigelingen of kinderleeftijd",
    "Persoonlijkheidsstoornissen",
    "Psychische stoornissen door een somatische aandoening",
    "Schizofrenie en andere psychotische stoornissen",
    "Somatisch-symptoomstoornis en verwante stoornissen",
    "Trauma- en stressorgerelateerde stoornissen",
    "Voedings- en eetstoornissen",]

def get_adm_dbc(dataset: int, frame ):
    dataset_end_date_column = {1: 'OntslagdatumTijd', 2: 'DateTimeCheckF', 3: 'DateTimeCheckP'}
    end_date = dataset_end_date_column[dataset]
    adm_dbc = frame.merge(dbc, how='inner', on='PseudoID')
    
    # Opnamedatum, Ontslagdatum, Startdatum, Einddatum
    adm_dbc["diagnosis_date"]= pd.to_datetime(adm_dbc["diagnosis_date"])
    
    adm_dbc['DbcWithin'] = adm_dbc.apply(lambda row: int(row.diagnosis_date <= row[end_date] and row.diagnosis_date >= row.OpnamedatumTijd), 1)
    
    # selecting only the rows that are DbcWithin
    dbc_within = adm_dbc[adm_dbc.DbcWithin != 0]
    
    # create columns of interesting data, this could not be dont in one go
    dbc_within2 = dbc_within[["OpnameID", "hoofddiagnose_groep"]]
    
    # only select with a diagnoses
    dbc_within2 = dbc_within2[dbc_within2["hoofddiagnose_groep"] != "Lege hoofddiagnose"]
    dbc_within2 = dbc_within2.drop_duplicates()
    
    # create columns of the values of hoofddiagnose_groep
    diagnoses = pd.get_dummies(dbc_within2["hoofddiagnose_groep"])

    dbc_within3 = pd.concat([dbc_within2, diagnoses], axis=1)
    
    if "Overige psychische stoornissen" not in dbc_within3:
            dbc_within3["Overige psychische stoornissen"] = np.nan
            
    dbc_hd = pd.DataFrame(dbc_within3.groupby('OpnameID')[hoofddiagnoses].max())
    
    # mvpi and persoonlijkheidsstoornis
    dbc_mp = pd.DataFrame(dbc_within.groupby('OpnameID')[['MeervoudigeProblematiekInd', 
                                                          'persoonlijkheidsstoornis']].max().reset_index())
    
    # zorgvraagtevraag
    dbc_zv = pd.DataFrame(dbc_within.assign(ZorgvraagzwaarteMin = dbc_within['zorgvraagzwaarte'].abs(),
                                            ZorgvraagzwaarteMax = dbc_within['zorgvraagzwaarte'].abs())
                          .groupby('OpnameID')
                          .agg({'ZorgvraagzwaarteMin':'min','ZorgvraagzwaarteMax':'max'}).reset_index())
    assert len(dbc_mp) == dbc_within.OpnameID.nunique() and len(dbc_zv) == len(dbc_mp)
    
    #merge interesting columns
    ds_dbc = dbc_mp.merge(dbc_zv, how="inner", on="OpnameID")
    
    #check that you didn't insert unnecessary rows
    assert len(dbc_mp) == len(ds_dbc)
    
    #merge hoofddiagnoses
    ds1_dbc = ds_dbc.merge(dbc_hd, how="inner", on="OpnameID")
    
    #merge interesting columns with adm_map
    adm_dbc = frame.merge(ds1_dbc, how= "left", on="OpnameID")
    
    #check that you still have the same number of admissions
    assert len(adm_dbc) == len(frame)
    
    #fill na values
    adm_dbc = adm_dbc.replace(np.nan,0)
    
    # test for missing values
    assert np.sum(adm_dbc.isnull().sum().values) == 0
    return adm_dbc


In [ ]:
# merge dbc based on date contraints with adm_map1
adm_dbc1 = get_adm_dbc(1,adm_map1)
adm_dbc1

In [ ]:
# merge dbc based on date contraints with adm_map2
adm_dbc2 = get_adm_dbc(2,adm_map2)
adm_dbc2

In [ ]:
# merge dbc based on date contraints with adm_map3
adm_dbc3 = get_adm_dbc(3,adm_map3)
adm_dbc3

In [ ]:
# merge administerings dataset 1

In [ ]:
len(administering), administering.PseudoID.nunique()

In [ ]:
# create past and future tranq prescriptions
# past 3/14 days and its the same in the karin dataset

var_pat = ["PseudoID",
        #"AfdelingOmschrijving",
        "Spoed",
        "EersteOpname",
        "Geslacht",
        "Leeftijd_opname",
        "Duur",
        "Leeftijd_startdatum_dossier",
        "incidents_during_admission",
        "incidents_before_admission",
        "MeervoudigeProblematiekInd",
        "persoonlijkheidsstoornis",
        "ZorgvraagzwaarteMin",
        "ZorgvraagzwaarteMax",
        "DoseDiazepam",
        "DoseDiazepamPre",
        "DoseDiazepamPost",   
]

UsefullVariables = var_pat + afd + hoofddiagnoses

# left join on PseudoID
# has no end date restriction for the post prescriptions
def get_adm_dbc(dataset: int, frame ):
    dataset_end_date_column = {1: 'OntslagdatumTijd', 2: 'DateTimeCheckF', 3: 'DateTimeCheckP'}
    end_date = dataset_end_date_column[dataset]
    adm_adm1 = frame.merge(administering[["PseudoID", "ToedienDatumTijd", "DoseDiazepam" ]], how="left", on="PseudoID")  

# remove rows where the ToedienDatumTijd is outside of the OpnamedatumTijd and OntslagdatumTijd
    adm_adm1 = adm_adm1[adm_adm1["ToedienDatumTijd"] >= adm_adm1["OpnamedatumTijd"]]
    adm_adm1 = adm_adm1[adm_adm1["ToedienDatumTijd"] <= adm_adm1["OntslagdatumTijd"]]

# create past and future prescriptions
    adm_adm1["Past"] = adm_adm1.apply(lambda row: int(row.ToedienDatumTijd <= row[end_date] and row.ToedienDatumTijd >= row.OpnamedatumTijd), 1)

# create two week prescriptions    
#    adm_adm1["TwoWeek"] = adm_adm1.apply(lambda row: int(row.ToedienDatumTijd <= row[end_date]+14 and row.ToedienDatumTijd >= row[end_date]), 1)
    
# diazepam prescribed in the past
    adm_adm1["DoseDiazepamPre"] = np.where(adm_adm1["Past"]== 1, adm_adm1["DoseDiazepam"], 0 )

# diazepam prescribed in the twoweek
 #   adm_adm1["DoseDiazepamTwoWeek"] = np.where(adm_adm1["TwoWeek"]== 1, adm_adm1["DoseDiazepam"], 0 )    
    
# diazepam prescribed in the future
    adm_adm1["DoseDiazepamPost"] = np.where(adm_adm1["Past"]== 0, adm_adm1["DoseDiazepam"], 0 )
    
# groupby
    adm_adm2 = pd.DataFrame(adm_adm1.groupby(["OpnameID", "OpnamedatumTijd", "OntslagdatumTijd"])[["DoseDiazepam","DoseDiazepamPre","DoseDiazepamPost"  ]].sum())                   
                                      
    Dataset = frame.merge(adm_adm2, on=["OpnameID", "OpnamedatumTijd","OntslagdatumTijd"], how="left")

#na values should be 0 because there are no prescriptions for tranquilizers
    Dataset["DoseDiazepam"] = Dataset["DoseDiazepam"].replace(np.nan, 0, regex=True)
    Dataset["DoseDiazepamPre"] = Dataset["DoseDiazepamPre"].replace(np.nan, 0, regex=True)
    Dataset["DoseDiazepamPost"] = Dataset["DoseDiazepamPost"].replace(np.nan, 0, regex=True)

# true dataset 
    DatasetWhole = Dataset[
    UsefullVariables
]
    assert np.sum(DatasetWhole.isnull().sum().values) == 0
    return DatasetWhole

In [ ]:
DatasetWhole = get_adm_dbc(1 , adm_dbc1)

del DatasetWhole["DoseDiazepamPost"]
del DatasetWhole["DoseDiazepamPre"]

DatasetWhole#.isnull().sum()

In [ ]:
Dataset14Days = get_adm_dbc(2 , adm_dbc2)

del Dataset14Days["DoseDiazepam"]

Dataset14Days#.isnull().sum()

In [ ]:
Dataset3Days = get_adm_dbc(3 , adm_dbc3)

del Dataset3Days["DoseDiazepam"]

Dataset3Days#.isnull().sum()

In [ ]:
# Load the previously assembled 14-day dataset
old_14_day_dataset = pd.read_csv(DATA_DIR + 'Dataset14Days.csv')
assert len(old_14_day_dataset) == len(Dataset14Days)
assert set([el for el in Dataset14Days.columns]) == set([el for el in old_14_day_dataset.columns])

In [ ]:
old_dict, new_dict = [{lbl: len(grp) for lbl, grp in df.groupby('PseudoID')} for df in (old_14_day_dataset, Dataset14Days)]

In [ ]:
assert len(old_dict) == len(new_dict)

In [ ]:
assert old_dict.keys() == new_dict.keys()

In [ ]:
assert len([el for el in old_dict.keys() if old_dict[el] != new_dict[el]]) == 0

In [ ]:
Dataset14Days['PabloID'] = Dataset14Days.apply(lambda row: '_'.join([str(int(row[el])) for el in ('PseudoID', 'Duur', 'Leeftijd_opname', 'EersteOpname', 'Klin.Acuut & Intensieve. Zorg Jeugd', 'DoseDiazepamPre')]), 1)
old_14_day_dataset['PabloID'] = old_14_day_dataset.apply(lambda row: '_'.join([str(int(row[el])) for el in ('PseudoID', 'Duur', 'Leeftijd_opname', 'EersteOpname', 'Klin.Acuut & Intensieve. Zorg Jeugd', 'DoseDiazepamPre')]), 1)
assert len(Dataset14Days) == Dataset14Days['PabloID'].nunique()
assert len(old_14_day_dataset) == old_14_day_dataset['PabloID'].nunique()

In [ ]:
merged = Dataset14Days.merge(old_14_day_dataset, on='PabloID', how='outer')

In [ ]:
assert len(merged) == len(Dataset14Days)
assert len(merged) == len(old_14_day_dataset)

In [ ]:
special_columns = ['DoseDiazepamPre', 'DoseDiazepamPost']
for col in special_columns:
    assert len(merged[merged.apply(lambda row: row[col + '_x'] == row[col + '_y'] or abs(row[col + '_x'] - row[col + '_y']) * 2 / (row[col + '_x'] + row[col + '_y']) < 0.005, 1)]) == len(merged)

In [ ]:
for col in columns:
    if col in special_columns:
        continue
    assert len(merged[merged.apply(lambda row: row[col + '_x'] != row[col + '_y'], 1)]) == 0, col

If we've gotten this far, then this notebook reproduces Jesse's dataset